In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet").load("s3://bronzebucketkrsna/products")
df = df.drop("_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
select * from products

In [0]:
%sql
create or replace function databricks_cata.bronze.discount_func(p_price double)
returns double
language sql
return p_price*0.90

In [0]:
%sql
select price, databricks_cata.bronze.discount_func(price) from products

In [0]:
discountedDf = df.withColumn("discount", expr("databricks_cata.bronze.discount_func(price)"))
discountedDf.display()

In [0]:
%sql
create or replace function databricks_cata.bronze.upper_func(p_brand String)
returns string
language python
as
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_cata.bronze.upper_func(brand) from products

In [0]:
df.write.format("delta").mode("overwrite").option("path", "s3://silverbucketkrsna/Retail_YT_Project/products/").save()


In [0]:
%sql
CREATE table databricks_cata.silver.products
USING DELTA
LOCATION 's3://silverbucketkrsna/Retail_YT_Project/products/';